Run the cell below before beginning this lab!

In [1]:
import requests
import pandas as pd
from pandas.io.json import json_normalize
import time
import os
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import math
import statistics

def get_price_hist(ticker,period,key):
    time.sleep(1)
    endpoint = 'https://api.tdameritrade.com/v1/marketdata/'+ticker+'/pricehistory'

    ##Define Payload
    payload = {'apikey': key,
    'periodType': 'year',
    'peirod':period,
    'frequencyType':'daily'}

    ### make request
    try:
        content = requests.get(url = endpoint, params = payload)
    except:
        print('API error, please review.')
        
    ### Convert to dictionary
    dictlist = []
    data = content.json()

    for key, value in data.items():
        temp = [key,value]
        dictlist.append(temp)
        
    try:
        hist_data = pd.DataFrame(dictlist[0][1])
        hist_data['ticker'] = ticker
        hist_data['datetime'] = pd.to_datetime(hist_data['datetime'],unit='ms')
        return hist_data
    except:
        df = pd.DataFrame()
        return df
    

def get_fundamental_from_td(ticker,key):
    time.sleep(1)
    endpoint = 'https://api.tdameritrade.com/v1/instruments'
    projection = 'fundamental'

    ##Define Payload
    payload = {'apikey': key,
               'symbol' : ticker,
                'projection': projection,
                }
    
    ### make request
    try:
        content = requests.get(url = endpoint, params = payload)
    except:
        print('API error, please review.')
        
    ### Convert to dictionary
    dictlist = []
    data = content.json()
    for key, value in data.items():
        temp = [key,value]
        dictlist.append(temp)
        
    try:
        df = pd.DataFrame(dictlist[0][1]).T.reset_index(drop=True).iloc[0]
        return df
    except:
        print(dictlist)
        df = pd.DataFrame()
        print(ticker + " not valid.")
        return df

def scrub_fundamental_data(tickers,key):
    master = pd.DataFrame()
    count = 1
    for ticker in tickers:
        temp = get_fundamental_from_td(ticker,key)
        temp = pd.DataFrame(temp).T
        hist_data = get_annual_returns([ticker],key)
        try:
            temp.columns = ['beta', 'bookValuePerShare', 'currentRatio', 'divGrowthRate3Year',
           'dividendAmount', 'dividendDate', 'dividendPayAmount',
           'dividendPayDate', 'dividendYield', 'epsChange', 'epsChangePercentTTM',
           'epsChangeYear', 'epsTTM', 'grossMarginMRQ', 'grossMarginTTM', 'high52',
           'interestCoverage', 'low52', 'ltDebtToEquity', 'marketCap',
           'marketCapFloat', 'netProfitMarginMRQ', 'netProfitMarginTTM',
           'operatingMarginMRQ', 'operatingMarginTTM', 'pbRatio', 'pcfRatio',
           'peRatio', 'pegRatio', 'prRatio', 'quickRatio', 'returnOnAssets',
           'returnOnEquity', 'returnOnInvestment', 'revChangeIn', 'revChangeTTM',
           'revChangeYear', 'sharesOutstanding', 'shortIntDayToCover',
           'shortIntToFloat', 'ticker', 'totalDebtToCapital', 'totalDebtToEquity',
           'vol10DayAvg', 'vol1DayAvg', 'vol3MonthAvg']
            temp = pd.merge(temp, hist_data, on='ticker')
        except:
            continue
        
        master = master.append(temp).reset_index(drop=True)
        count+= 1
    return master

def scrub_price_hist(tickers,key):
    master = pd.DataFrame()
    for ticker in tickers:
        price_data = get_price_hist(ticker,1,key)
        master = master.append(price_data, ignore_index=True)
    return master

def get_annual_returns(tickers,key):
    master = pd.DataFrame()
    for ticker in tickers:
        try:
            price_data = get_price_hist(ticker,1,key).iloc[::-1]
            return_percent = price_data['close'].iloc[0] / price_data['close'].iloc[-1] -1
            df = pd.DataFrame({"ticker":ticker,  
                               "Annual_Return_Percent":return_percent},
                               index = [0])
            master = master.append(df, ignore_index=True)
        except:

            print('error with', ticker)
    return master

1. Pull the 1 year price history using the scrub_price_hist function of any stock you like.

In [8]:
hist=scrub_price_hist('GOOG',key)
hist

,open,high,low,close,volume,datetime,ticker
0,44.22,44.755,44.22,44.41,487687,2021-04-14 05:00:00,G
1,44.65,44.960,44.43,44.56,683831,2021-04-15 05:00:00,G
2,44.83,45.210,44.65,44.73,2035127,2021-04-16 05:00:00,G
3,44.72,44.935,44.33,44.52,493783,2021-04-19 05:00:00,G
4,44.17,44.810,44.17,44.72,875370,2021-04-20 05:00:00,G
...,...,...,...,...,...,...,...
1015,43.25,44.080,43.22,43.66,684763,2022-04-08 05:00:00,G
1016,43.41,44.030,43.23,43.25,918839,2022-04-11 05:00:00,G
1017,43.75,43.940,42.62,42.62,717646,2022-04-12 05:00:00,G
1018,42.49,43.010,42.14,43.00,552876,2022-04-13 05:00:00,G


2. Clean up the output so it mirrors our output for Apple.

In [9]:
hist = hist[['close','datetime','ticker']]
hist = hist.sort_values(by = 'datetime', ascending = False)
hist['GOOG_Return_%'] = hist['close'] / hist['close'].shift(-1) - 1
hist.rename(columns={'close': 'GOOG_Close'}, inplace=True)
hist.drop(columns = ['ticker'], inplace = True)
hist

,GOOG_Close,datetime,GOOG_Return_%
1019,42.12,2022-04-14 05:00:00,-0.419915
764,72.61,2022-04-14 05:00:00,0.000000
509,72.61,2022-04-14 05:00:00,0.723884
254,42.12,2022-04-14 05:00:00,-0.020465
253,43.00,2022-04-13 05:00:00,-0.404350
...,...,...,...
766,44.56,2021-04-15 05:00:00,0.003378
0,44.41,2021-04-14 05:00:00,0.000000
765,44.41,2021-04-14 05:00:00,-0.320532
255,65.36,2021-04-14 05:00:00,0.000000


3.  Think of another market proxy besides the S&P 500, pull it's data, and clean it up.

In [10]:
hist2=scrub_price_hist('GOOG',key)
hist2 = hist2[['close','datetime','ticker']]
hist2 = hist2.sort_values(by = 'datetime', ascending = False)
hist2['BND_Return_%'] = hist2['close'] / hist2['close'].shift(-1) - 1
hist2.rename(columns={'close': 'BND_Close'}, inplace=True)
hist2.drop(columns = ['ticker'], inplace = True)
hist2

,BND_Close,datetime,BND_Return_%
1019,42.12,2022-04-14 05:00:00,-0.419915
764,72.61,2022-04-14 05:00:00,0.000000
509,72.61,2022-04-14 05:00:00,0.723884
254,42.12,2022-04-14 05:00:00,-0.020465
253,43.00,2022-04-13 05:00:00,-0.404350
...,...,...,...
766,44.56,2021-04-15 05:00:00,0.003378
0,44.41,2021-04-14 05:00:00,0.000000
765,44.41,2021-04-14 05:00:00,-0.320532
255,65.36,2021-04-14 05:00:00,0.000000


4. Merge the two data sets, so that it mirrors the 'consol_data' variable.  Then merge in the new proxy data and SPY data.

In [11]:
# Run this cell first
spy_data = scrub_price_hist(['SPY'],key)
spy_data = spy_data[['close','datetime','ticker']]
spy_data = spy_data.sort_values(by = 'datetime', ascending = False)
spy_data['SPY_Return_%'] = spy_data['close'] / spy_data['close'].shift(-1) - 1
spy_data.rename(columns={'close': 'SPY_Close'}, inplace=True)
spy_data.drop(columns = ['ticker'], inplace = True)
spy_data.head()

,SPY_Close,datetime,SPY_Return_%
254,437.79,2022-04-14 05:00:00,-0.012452
253,443.31,2022-04-13 05:00:00,0.011454
252,438.29,2022-04-12 05:00:00,-0.003705
251,439.92,2022-04-11 05:00:00,-0.017092
250,447.57,2022-04-08 05:00:00,-0.002674


In [12]:
consol_data = pd.merge(hist,hist2,on=['datetime'], how = 'left').dropna()
consol_data = pd.merge(consol_data,spy_data,on=['datetime'], how = 'left').dropna()
consol_data

,GOOG_Close,datetime,GOOG_Return_%,BND_Close,BND_Return_%,SPY_Close,SPY_Return_%
0,42.12,2022-04-14 05:00:00,-0.419915,42.12,-0.419915,437.79,-0.012452
1,42.12,2022-04-14 05:00:00,-0.419915,72.61,0.000000,437.79,-0.012452
2,42.12,2022-04-14 05:00:00,-0.419915,72.61,0.723884,437.79,-0.012452
3,42.12,2022-04-14 05:00:00,-0.419915,42.12,-0.020465,437.79,-0.012452
4,72.61,2022-04-14 05:00:00,0.000000,42.12,-0.419915,437.79,-0.012452
...,...,...,...,...,...,...,...
4059,66.92,2021-04-15 05:00:00,0.501795,44.56,0.003378,415.87,0.010742
4060,44.56,2021-04-15 05:00:00,0.003378,44.56,-0.334130,415.87,0.010742
4061,44.56,2021-04-15 05:00:00,0.003378,66.92,0.000000,415.87,0.010742
4062,44.56,2021-04-15 05:00:00,0.003378,66.92,0.501795,415.87,0.010742


5. Calculate the beta of your new stock relative to both SPY and your new market proxy.  Are they different significantly?
   Consider why they may be different.

In [13]:
##Now calc beta using statsmodels
x = consol_data[['SPY_Return_%']]
y = consol_data[['GOOG_Return_%']]
z = consol_data[['BND_Return_%']]

model = sm.OLS(y, x)
results = model.fit()
print('Beta GOOG, SPY: '+str(round(results.params[0],2)))

model = sm.OLS(y, z)
results = model.fit()
print('Beta GOOG, BND: '+str(round(results.params[0],2)))

Beta GOOG, SPY: 0.04
Beta GOOG, BND: 0.07


6. Pick a stock we haven't looked at and grab its fundamental data.

In [14]:
get_fundamental_from_td('VXUS',key)

beta                                         0
bookValuePerShare                            0
currentRatio                                 0
divGrowthRate3Year                           0
dividendAmount                          0.4016
dividendDate           2022-03-21 00:00:00.000
dividendPayAmount                       0.1004
dividendPayDate        2022-03-24 00:00:00.000
dividendYield                             0.69
epsChange                                    0
epsChangePercentTTM                          0
epsChangeYear                                0
epsTTM                                       0
grossMarginMRQ                               0
grossMarginTTM                               0
high52                                   67.51
interestCoverage                             0
low52                                   55.145
ltDebtToEquity                               0
marketCap                                    0
marketCapFloat                               0
netProfitMarg

7. Calculate the annual volatility of TQQQ.

In [15]:
tqqq=scrub_price_hist('TQQQ',key)
tqqq = tqqq[['close','datetime','ticker']]
tqqq = tqqq.sort_values(by = 'datetime', ascending = False)
tqqq['TQQQ_Return_%'] = tqqq['close'] / tqqq['close'].shift(-1) - 1
tqqq.rename(columns={'close': 'TQQQ_Close'}, inplace=True)
tqqq.drop(columns = ['ticker'], inplace = True)

tqqq_vol = math.sqrt(pd.DataFrame([tqqq['TQQQ_Return_%'],tqqq['TQQQ_Return_%']]).T.cov().iloc[0,0])*math.sqrt(252)
print('TQQQ Volatility: '+str(round(tqqq_vol*100,2))+'%')

TQQQ Volatility: 23.47%


8. Find the beta to SPY of AMD as well as its volatility and make a conclusion about its risk profile.

In [19]:
hist = scrub_price_hist('AMD',key)
hist = hist[['close','datetime','ticker']]
hist = hist.sort_values(by = 'datetime', ascending = False)
hist['AMD_Return_%'] = hist['close'] / hist['close'].shift(-1) - 1
hist.rename(columns={'close': 'AMD_Close'}, inplace=True)
hist.drop(columns = ['ticker'], inplace = True)

consol_data = pd.merge(hist,spy_data,on=['datetime'], how = 'left').dropna()

##Now calc beta using statsmodels
x = consol_data[['SPY_Return_%']]
y = consol_data[['AMD_Return_%']]

model = sm.OLS(y, x)
results = model.fit()
print('Beta AMD, SPY: '+str(round(results.params[0],2)))

amd_vol = math.sqrt(pd.DataFrame([hist['AMD_Return_%'],hist['AMD_Return_%']]).T.cov().iloc[0,0])*math.sqrt(252)
print('AMD Volatility: '+str(round(amd_vol*100,2))+'%')

Beta AMD, SPY: -3.31
AMD Volatility: 3619.02%


9. Get the fundamental data of AMC.  

In [20]:
get_fundamental_from_td('AMC',key)

beta                                   1.59637
bookValuePerShare                            0
currentRatio                           1.04609
divGrowthRate3Year                           0
dividendAmount                               0
dividendDate                                  
dividendPayAmount                          0.2
dividendPayDate        2019-12-16 00:00:00.000
dividendYield                                0
epsChange                                    0
epsChangePercentTTM                    93.0757
epsChangeYear                          95.7839
epsTTM                                       0
grossMarginMRQ                         68.4817
grossMarginTTM                         70.5052
high52                                   72.62
interestCoverage                       0.23068
low52                                      8.9
ltDebtToEquity                               0
marketCap                              9312.36
marketCapFloat                         463.598
netProfitMarg

10. Find the annualized volatility of an equally weighted portfolio of AMGN and PG.  Assume 252 trading days in a year and use one year of historicals.

In [24]:
hist = scrub_price_hist('AMGN',key)
hist = hist[['close','datetime','ticker']]
hist = hist.sort_values(by = 'datetime', ascending = False)
hist['AMGN_Return_%'] = hist['close'] / hist['close'].shift(-1) - 1
hist.rename(columns={'close': 'AMGN_Close'}, inplace=True)
hist.drop(columns = ['ticker'], inplace = True)

hist2 = scrub_price_hist('PG',key)
hist2 = hist2[['close','datetime','ticker']]
hist2 = hist2.sort_values(by = 'datetime', ascending = False)
hist2['PG_Return_%'] = hist2['close'] / hist2['close'].shift(-1) - 1
hist2.rename(columns={'close': 'PG_Close'}, inplace=True)
hist2.drop(columns = ['ticker'], inplace = True)

vol = math.sqrt(pd.DataFrame([hist['AMGN_Return_%']*.5,hist2['PG_Return_%']*.5]).T.cov().iloc[0,0])*math.sqrt(252)
print('Equal Volatility: '+str(round(vol*100,2))+'%')

Equal Volatility: 1777.2%


11. Find the annualized volatility of AMGN and PG respecively with the same assumptions as question 10.

In [25]:
vol = math.sqrt(pd.DataFrame([hist['AMGN_Return_%'],hist2['PG_Return_%']]).T.cov().iloc[0,0])*math.sqrt(252)
print('Equal Volatility: '+str(round(vol*100,2))+'%')

Equal Volatility: 3554.41%


12. Find the volatility of XRX using data from June of 2021 only.  Compare it to its volatility in September of 2021.

In [31]:
hist = scrub_price_hist('XRX',key)
hist = hist[['close','datetime','ticker']]
hist = hist.sort_values(by = 'datetime', ascending = False)
hist['XRX_Return_%'] = hist['close'] / hist['close'].shift(-1) - 1
hist.rename(columns={'close': 'XRX_Close'}, inplace=True)
hist.drop(columns = ['ticker'], inplace = True)

jun=hist[(hist['datetime'].dt.month==6)&(hist['datetime'].dt.year==2021)]
sep=hist[(hist['datetime'].dt.month==9)&(hist['datetime'].dt.year==2021)]

vol = math.sqrt(pd.DataFrame([jun['XRX_Return_%'],sep['XRX_Return_%']]).T.cov().iloc[0,0])*math.sqrt(252)
print('Volatility: '+str(round(vol*100,2))+'%')

Volatility: 1869.16%


13.  Find the 1-year value at risk for a portfolio of 25% FB and 75% MO.  Assume 252 trading days in a year, and use the most recent 1 year pricing data.

In [34]:
###Define Portfolio Inputs
fb_weight = .25
mo_weight = .75
portfolio_value = 100000
days_into_future = 252
tickers = ['FB','MO']

#Let's calculate our portfolio volatility
###Grab Price History
price_hist = scrub_price_hist(tickers,key)
price_hist = price_hist[['close','ticker','datetime']]


###Calculate Portfolio Volatility
count = 0
for ticker in tickers:
    if count == 0:
        temp = price_hist[price_hist['ticker'] == ticker]
        temp[ticker +'_Return_%'] = temp['close'] / temp['close'].shift(1) - 1
        temp = temp.dropna()
        temp = temp.drop(['close','ticker'], axis = 1)
        master = temp
        count = 1
    else:
        temp = price_hist[price_hist['ticker'] == ticker]
        temp[ticker +'_Return_%'] = temp['close'] / temp['close'].shift(1) - 1
        temp = temp.dropna()
        temp = temp.drop(['close','ticker'], axis = 1)
        master = pd.merge(master,temp,how = 'left',on = ['datetime'])

consol_returns = pd.DataFrame([master[tickers[0] + '_Return_%']*fb_weight,
                               master[tickers[1] + '_Return_%']*mo_weight]).T
port_volatility = math.sqrt(consol_returns.cov().to_numpy().sum())*math.sqrt(days_into_future)

print('Portfolio Volatility: ',round(port_volatility*100,2),'%')

###Calculate VaR
VaR = portfolio_value * 1.65 * port_volatility
print('$',round(VaR,2))

Portfolio Volatility:  18.97 %
$ 31301.15


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


## -------------EXTEND YOUR UNDERSTANDING-------------

Assume your broker allows you to invest on margin with no cost of borrowing up to 30% (i.e. for each \\$1 you have, you can invest \\$1.30).  Assume you have $10,000, want to use all the margin, and are feeling very good about an equally weighted portfolio of AIV and AIRC.  What is your one-month VaR?  Assume 21 trading days in a month, and use the most recent year's worth of data to estimate volatility.  

In [35]:
###Define Portfolio Inputs
aiv_weight = .5
airc_weight = .5
portfolio_value = 10000
days_into_future = 21
tickers = ['AIV','AIRC']

#Let's calculate our portfolio volatility
###Grab Price History
price_hist = scrub_price_hist(tickers,key)
price_hist = price_hist[['close','ticker','datetime']]


###Calculate Portfolio Volatility
count = 0
for ticker in tickers:
    if count == 0:
        temp = price_hist[price_hist['ticker'] == ticker]
        temp[ticker +'_Return_%'] = temp['close'] / temp['close'].shift(1) - 1
        temp = temp.dropna()
        temp = temp.drop(['close','ticker'], axis = 1)
        master = temp
        count = 1
    else:
        temp = price_hist[price_hist['ticker'] == ticker]
        temp[ticker +'_Return_%'] = temp['close'] / temp['close'].shift(1) - 1
        temp = temp.dropna()
        temp = temp.drop(['close','ticker'], axis = 1)
        master = pd.merge(master,temp,how = 'left',on = ['datetime'])

consol_returns = pd.DataFrame([master[tickers[0] + '_Return_%']*aiv_weight,
                               master[tickers[1] + '_Return_%']*airc_weight]).T
port_volatility = math.sqrt(consol_returns.cov().to_numpy().sum())*math.sqrt(days_into_future)

print('Portfolio Volatility: ',round(port_volatility*100,2),'%')

###Calculate VaR
VaR = portfolio_value * 1.3 * 1.65 * port_volatility
print('$',round(VaR,2))

Portfolio Volatility:  6.92 %
$ 1484.05


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
